In [1]:
import os
import jieba
import pyltp
import pandas as pd
from gensim.models.word2vec import Word2Vec

## Initialize

In [2]:
LTP_DATA_DIR = '../data/ltp_data_v3.4.0'  # ltp模型目录的路径
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')
pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')
ner_model_path = os.path.join(LTP_DATA_DIR, 'ner.model')
par_model_path = os.path.join(LTP_DATA_DIR, 'parser.model')
srl_model_path = os.path.join(LTP_DATA_DIR, 'pisrl.model')

In [3]:
segmentor = pyltp.Segmentor()
segmentor.load(cws_model_path)

postagger = pyltp.Postagger()
postagger.load(pos_model_path)

recognizer = pyltp.NamedEntityRecognizer()
recognizer.load(ner_model_path)

parser = pyltp.Parser()
parser.load(par_model_path)

labeller = pyltp.SementicRoleLabeller() # 初始化实例
labeller.load(srl_model_path)  # 加载模型

In [4]:
# news path
NEWS_DATA_PATH = '../data/corpus/sqlResult_1558435.csv'
df = pd.read_csv(NEWS_DATA_PATH, encoding='gb18030')

In [5]:
# Word2Vec Model
NEWS_WORD2VEC_MODEL_PATH = '../data/corpus/news-model.model'
news_model = Word2Vec.load(NEWS_WORD2VEC_MODEL_PATH)

## Similiar words for '说'

In [6]:
say_words = news_model.wv.most_similar('说', topn=20)

In [7]:
say_words_dict = dict(say_words)

In [8]:
say_words_dict['说'] = 1

In [9]:
say_words_dict

{'表示': 0.8519119024276733,
 '指出': 0.7626331448554993,
 '坦言': 0.7605428695678711,
 '告诉': 0.7429102063179016,
 '认为': 0.7411446571350098,
 '看来': 0.6876845955848694,
 '称': 0.6868260502815247,
 '文说': 0.6837325096130371,
 '明说': 0.6574992537498474,
 '介绍': 0.6380150318145752,
 '强调': 0.6211819052696228,
 '透露': 0.6170471906661987,
 '所说': 0.615271270275116,
 '中说': 0.608891487121582,
 '说道': 0.5734717845916748,
 '时说': 0.5723824501037598,
 '地说': 0.5509226322174072,
 '写道': 0.5426820516586304,
 '问': 0.5383782386779785,
 '提到': 0.5280981659889221,
 '说': 1}

## Extration Example

In [10]:
contents = df[~df['content'].isnull()]['content']

In [11]:
news = contents[96]

In [12]:
news

'中新网6月23日电 (记者潘旭临) 意大利航空首席商务官乔治先生22日在北京接受中新网记者专访时表示，意航确信中国市场对意航的重要性，目前意航已将发展中国市场提升到战略层级的高度，未来，意航将加大在华布局，提升业务水平。\r\n到意大利航空履职仅7个月的乔治，主要负责包括中国市场在内的亚太业务。此次北京之行，目标是加深对中国市场的了解，寻找合作和投资良机。\r\n他说，“自己是第一次访问北京，也是第一次到中国访问。虽然此次时间短暂，但中国传统文化、古代建筑以及可口的中餐等，都给他留下非常美好的印象。”\r\n自从去年7月，意大利航空首次开启罗马至北京往返直飞航线后，目前每周有4个航班，提供近千个座位，业绩状况一直稳步有升。乔治称，随着对华业务不断提升，意航明年可能会将每周4班提高到每天一班。同时，意航会借罗马新航站楼启用之际，吸引更多中国旅客到意大利旅游和转机。此外，还将加大对北京直飞航线的投资，如翻新航班座椅，增加电视中有关中国内容的娱乐节目、提高机上中文服务、餐饮服务、完善意航中文官方网站，提升商务舱和普通舱的舒适度等。\r\n他表示，随着中国经济持续稳步增长，中国将很快跃居成为世界第一出境旅游大国。去年中国出境旅游突破了一亿人次，来意大利旅游的中国人也超过百万人次，今年还会增加。在面对如此巨大的旅游市场时，作为本土老牌航空公司，意航只有练好“内功”，不断完善和提高自身业务能力，才能在市场竞争中，占得先机，立于不败之地。\r\n在谈到目前意大利航空的经营状况时，乔治先生坦言，走过70年的意大利航空，目前正在经历如裁员、管理层重组等历史上又一次转型期。 乔治表示，意航转型期也是投资者的机遇期，意大利航空已做好吸引全球业界投资的准备，以迅速扩充实力，目前已有33家国际投资机构向意航提出投资申请，能否接受、如何接受，意航需做全面评估才会决定。\r\n乔治表示，在转型期内，意航业务如常，资金充分，并计划在下半年和明年增加中远程航线的投资，尤其是稳固和加强像中国这样的大市场投资。\r\n'

In [13]:
words = segmentor.segment(news.replace('\r\n', ''))

In [14]:
postags = postagger.postag(words)

In [15]:
netags = recognizer.recognize(words, postags)

In [16]:
' '.join(netags)

'O O O O O O S-Nh O B-Ni I-Ni E-Ni O S-Nh O O O S-Ns O O O O O O O O O S-Ns O O O O O O O O O O O S-Ns O O O O O O O O O O O O O O O O O O O O O O S-Ns O O O O O O O S-Nh O O O O S-Ns O O O O O O O O S-Ns O O O O O O O S-Ns O O O O O O O O O O O O O O O O O O O S-Ns O O O O O O S-Ns O O O O O O O O S-Ns O O O O O O O O O O O O O O O O O O O O O O O O O S-Ns O O O O S-Ns O S-Ns O O O O O O O O O O O O O O O O O O O O O O O O O O S-Nh O O O O O O O O O O O O O O O O O O O O O O O O O O O O B-Ni E-Ni O O O O O O O S-Ns O O S-Ns O O O O O O O O O O S-Ns O O O O O O O O O O O O O O O S-Ns O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O S-Ns O O O O O S-Ns O O O O O O O O O O O S-Ns O O O O O O O O S-Ns O O S-Ns O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O S-Ns O O O O O O S-Nh O O O O O O O S-Ns O O O O O O O O O O O O O O O O O O S-Nh O O O O O O O O O O S-Ns O O O O O O O O O O O O O O O O O O O O O O O

In [17]:
' '.join(words)

'中新网 6月 23日 电 ( 记者 潘旭临 ) 意大利 航空 首席 商务官 乔治 先生 22日 在 北京 接受 中新网 记者 专访 时 表示 ， 意航 确信 中国 市场 对 意航 的 重要性 ， 目前 意航 已 将 发展 中国 市场 提升 到 战略 层级 的 高度 ， 未来 ， 意航 将 加大 在 华 布局 ， 提升 业务 水平 。 到 意大利 航空 履职 仅 7 个 月 的 乔治 ， 主要 负责 包括 中国 市场 在内 的 亚 太 业务 。 此次 北京 之 行 ， 目标 是 加深 对 中国 市场 的 了解 ， 寻找 合作 和 投资 良机 。 他 说 ， “ 自己 是 第一 次 访问 北京 ， 也 是 第一 次 到 中国 访问 。 虽然 此次 时间 短暂 ， 但 中国 传统 文化 、 古代 建筑 以及 可口 的 中餐 等 ， 都 给 他 留下 非常 美好 的 印象 。 ” 自从 去年 7月 ， 意大利 航空 首 次 开启 罗马 至 北京 往返 直 飞 航线 后 ， 目前 每周 有 4 个 航班 ， 提供 近 千 个 座位 ， 业绩 状况 一直 稳步 有 升 。 乔治 称 ， 随着 对 华 业务 不断 提升 ， 意航 明年 可能 会 将 每周 4 班 提高 到 每天 一 班 。 同时 ， 意航 会 借 罗马 新航 站楼 启用 之际 ， 吸引 更 多 中国 旅客 到 意大利 旅游 和 转机 。 此外 ， 还 将 加大 对 北京 直 飞 航线 的 投资 ， 如 翻新 航班 座椅 ， 增加 电视 中 有关 中国 内容 的 娱乐 节目 、 提高 机上 中文 服务 、 餐饮 服务 、 完善 意航 中文 官方 网站 ， 提升 商务舱 和 普通舱 的 舒适度 等 。 他 表示 ， 随着 中国 经济 持续 稳步 增长 ， 中国 将 很快 跃居 成为 世界 第一 出境 旅游 大国 。 去年 中国 出境 旅游 突破 了 一亿 人次 ， 来 意大利 旅游 的 中国 人 也 超过 百万 人次 ， 今年 还 会 增加 。 在 面对 如此 巨大 的 旅游 市场 时 ， 作为 本土 老牌 航空 公司 ， 意航 只有 练 好 “ 内功 ” ， 不断 完善 和 提高 自身 业务 能力 ， 才 能 在 市场 竞争 中 ， 占 得 先机 ， 立于不败之地 。 在 谈 到 目前 意大利 航空 的 经营 状

In [18]:
' '.join(postags)

'n nt nt n wp n nh wp ns n n n nh n nt p ns v nz n v n v wp j v ns n p j u n wp nt j d p v ns n v v n n u n wp nt wp j d v p j n wp v n n wp v ns n n d m q n u nh wp d v v ns n u u j j n wp r ns u n wp n v v p ns n u v wp v v c v n wp r v wp wp r v m q v ns wp d v m q v ns v wp c r n a wp c ns n n wp nt n c a u n u wp d p r v d a u n wp wp p nt nt wp ns n m q v ns p ns v d v n nd wp nt r v m q n wp v a m q n wp n n d d v v wp nh v wp p p j n d v wp j nt v v p r m n v v r m q wp n wp j v v ns j n v nd wp v d a ns n v ns v c n wp c wp d d v p ns d v n u v wp v v n n wp v n nd p ns n u v n wp v nl nz v wp n v wp v j nz n n wp v n c n u n u wp r v wp p ns n v d v wp ns d d v v n m v v n wp nt ns v v v u m q wp v ns v u ns n d v m q wp nt d v v wp p v r a u v n n wp p n b n n wp j c v a wp n wp wp d v c v r n n wp c v p n v nd wp v v n wp i wp p v v nt ns n u v n n wp nh n v wp v m q u ns n wp nt d v v v wp n v u n nd d m q n wp nh v wp j n d v n u n wp ns n d v a v n v u v wp p a v n wp nt

In [19]:
print('\t'.join(['{}:{}/{}'.format(i, words[i], postags[i]) for i in range(len(words))]))

0:中新网/n	1:6月/nt	2:23日/nt	3:电/n	4:(/wp	5:记者/n	6:潘旭临/nh	7:)/wp	8:意大利/ns	9:航空/n	10:首席/n	11:商务官/n	12:乔治/nh	13:先生/n	14:22日/nt	15:在/p	16:北京/ns	17:接受/v	18:中新网/nz	19:记者/n	20:专访/v	21:时/n	22:表示/v	23:，/wp	24:意航/j	25:确信/v	26:中国/ns	27:市场/n	28:对/p	29:意航/j	30:的/u	31:重要性/n	32:，/wp	33:目前/nt	34:意航/j	35:已/d	36:将/p	37:发展/v	38:中国/ns	39:市场/n	40:提升/v	41:到/v	42:战略/n	43:层级/n	44:的/u	45:高度/n	46:，/wp	47:未来/nt	48:，/wp	49:意航/j	50:将/d	51:加大/v	52:在/p	53:华/j	54:布局/n	55:，/wp	56:提升/v	57:业务/n	58:水平/n	59:。/wp	60:到/v	61:意大利/ns	62:航空/n	63:履职/n	64:仅/d	65:7/m	66:个/q	67:月/n	68:的/u	69:乔治/nh	70:，/wp	71:主要/d	72:负责/v	73:包括/v	74:中国/ns	75:市场/n	76:在内/u	77:的/u	78:亚/j	79:太/j	80:业务/n	81:。/wp	82:此次/r	83:北京/ns	84:之/u	85:行/n	86:，/wp	87:目标/n	88:是/v	89:加深/v	90:对/p	91:中国/ns	92:市场/n	93:的/u	94:了解/v	95:，/wp	96:寻找/v	97:合作/v	98:和/c	99:投资/v	100:良机/n	101:。/wp	102:他/r	103:说/v	104:，/wp	105:“/wp	106:自己/r	107:是/v	108:第一/m	109:次/q	110:访问/v	111:北京/ns	112:，/wp	113:也/d	114:是/v	115:第一/m	116:次/q	117:到/v	118:中国/ns	119:访问/v	120:。/wp	121:虽然/c	122:此次/r	123:时间/n	1

## Extract '说' and similar words from content

In [30]:
say_positions = [(w, i) for i, w in enumerate(words) if w in say_words_dict]

In [31]:
say_positions

[('表示', 22),
 ('说', 103),
 ('称', 188),
 ('表示', 284),
 ('坦言', 386),
 ('表示', 412),
 ('表示', 468)]

## NER - extract all the names from the content

In [22]:
all_names_positions = [(i, tag) for i, tag in enumerate(netags) if 'Nh' in tag]

In [23]:
all_names_positions

[(6, 'S-Nh'),
 (12, 'S-Nh'),
 (69, 'S-Nh'),
 (187, 'S-Nh'),
 (384, 'S-Nh'),
 (411, 'S-Nh'),
 (467, 'S-Nh')]

In [24]:
all_names = [(words[name[0]], name[0]) for name in all_names_positions]

In [33]:
all_names

[('潘旭临', 6),
 ('乔治', 12),
 ('乔治', 69),
 ('乔治', 187),
 ('乔治', 384),
 ('乔治', 411),
 ('乔治', 467)]

In [35]:
names = set(map(lambda x: x[0], all_names))

In [36]:
names

{'乔治', '潘旭临'}

## 依存句法分析

In [26]:
arcs = parser.parse(words, postags)

In [32]:
print("\n".join("%d:%s=>%d:%s" % 
        (i, '********{}********'.format(words[i]) if words[i] in say_words_dict.keys() else words[i], arc.head, arc.relation) 
                for i, arc in enumerate(arcs)))

0:中新网=>4:ADV
1:6月=>3:ATT
2:23日=>4:ADV
3:电=>0:HED
4:(=>7:WP
5:记者=>7:ATT
6:潘旭临=>4:CMP
7:)=>7:WP
8:意大利=>10:ATT
9:航空=>11:ATT
10:首席=>12:ATT
11:商务官=>13:ATT
12:乔治=>14:ATT
13:先生=>23:SBV
14:22日=>18:ADV
15:在=>18:ADV
16:北京=>16:POB
17:接受=>22:ATT
18:中新网=>20:ATT
19:记者=>21:ATT
20:专访=>18:VOB
21:时=>23:ADV
22:********表示********=>4:VOB
23:，=>23:WP
24:意航=>26:SBV
25:确信=>23:VOB
26:中国=>28:ATT
27:市场=>32:ATT
28:对=>32:ATT
29:意航=>29:POB
30:的=>29:RAD
31:重要性=>26:VOB
32:，=>26:WP
33:目前=>37:ADV
34:意航=>37:SBV
35:已=>37:ADV
36:将=>26:COO
37:发展=>37:POB
38:中国=>40:ATT
39:市场=>41:SBV
40:提升=>38:VOB
41:到=>41:CMP
42:战略=>44:ATT
43:层级=>46:ATT
44:的=>44:RAD
45:高度=>41:VOB
46:，=>41:WP
47:未来=>52:ADV
48:，=>48:WP
49:意航=>52:SBV
50:将=>52:ADV
51:加大=>41:COO
52:在=>55:ATT
53:华=>53:POB
54:布局=>52:VOB
55:，=>52:WP
56:提升=>52:COO
57:业务=>59:ATT
58:水平=>57:VOB
59:。=>23:WP
60:到=>73:ADV
61:意大利=>63:ATT
62:航空=>64:ATT
63:履职=>70:ATT
64:仅=>68:ATT
65:7=>67:ATT
66:个=>68:ATT
67:月=>70:ATT
68:的=>68:RAD
69:乔治=>61:POB
70:，=>61:WP
71:主要=>73:ADV
72:负责=>23:COO
73:包括=>8

## Extract from name

In [75]:
def get_all_words_related_to(index):
    return [words[i] for i, arc in enumerate(arcs) if arc.head == index]

In [104]:
candidates = [(name, arcs[name[1]].relation, arcs[name[1]].head) 
              for name in all_names if arcs[name[1]].relation == 'SBV']

In [105]:
candidates

[(('乔治', 187), 'SBV', 189),
 (('乔治', 411), 'SBV', 413),
 (('乔治', 467), 'SBV', 469)]

In [38]:
def is_sentence_end(w):
    return w in ["。", "!", "！"]

In [111]:
for candidate in candidates:
    sentence = ''
    for w in words[candidate[2] + 1:]:
        sentence += w
        if is_sentence_end(w):
            print('{}:{}'.format(candidate[0][0], sentence))
            break

乔治:随着对华业务不断提升，意航明年可能会将每周4班提高到每天一班。
乔治:意航转型期也是投资者的机遇期，意大利航空已做好吸引全球业界投资的准备，以迅速扩充实力，目前已有33家国际投资机构向意航提出投资申请，能否接受、如何接受，意航需做全面评估才会决定。
乔治:在转型期内，意航业务如常，资金充分，并计划在下半年和明年增加中远程航线的投资，尤其是稳固和加强像中国这样的大市场投资。


## Extract from 'say' words

In [50]:
options = []
for say_word, pos in say_positions:
    option = {'say': say_word}
    # Get the name who say the words
    for i in range(pos, 0, -1):
        w = words[i]
        if w in names:
            option['name'] = w
            break
    
    sentence = ''
    for w in words[pos + 1:]:
        sentence += w
        if is_sentence_end(w):
            option['sentence'] = sentence
            break
    
    options.append(option)

In [51]:
for option in options:
    print('{} {} {}'.format(option['name'], option['say'], option['sentence']))

乔治 表示 ，意航确信中国市场对意航的重要性，目前意航已将发展中国市场提升到战略层级的高度，未来，意航将加大在华布局，提升业务水平。
乔治 说 ，“自己是第一次访问北京，也是第一次到中国访问。
乔治 称 ，随着对华业务不断提升，意航明年可能会将每周4班提高到每天一班。
乔治 表示 ，随着中国经济持续稳步增长，中国将很快跃居成为世界第一出境旅游大国。
乔治 坦言 ，走过70年的意大利航空，目前正在经历如裁员、管理层重组等历史上又一次转型期。
乔治 表示 ，意航转型期也是投资者的机遇期，意大利航空已做好吸引全球业界投资的准备，以迅速扩充实力，目前已有33家国际投资机构向意航提出投资申请，能否接受、如何接受，意航需做全面评估才会决定。
乔治 表示 ，在转型期内，意航业务如常，资金充分，并计划在下半年和明年增加中远程航线的投资，尤其是稳固和加强像中国这样的大市场投资。
